In [ ]:
import pandas as pd

# Load the datasets
file1_path = '2019 전국 종합.csv'


data1 = pd.read_csv(file1_path, encoding='utf-8')

# Start preprocessing to align data1 to the structure of data2
# Rename columns in data1 to match data2

data1 = data1.rename(columns={'지역': '시군구'})

## Clean '시군구' column: remove spaces and numbers
data1['시군구'] = data1['시군구'].str.replace(r'[\d\s]', '', regex=True)
# Replace entries in '시군구' that do not end with '시', '군', or '구' (excluding '세종') with '소계'
data1['시군구'] = data1['시군구'].apply(lambda x: x if x == '세종' or (x.endswith(('시', '군', '구')) and x != '대구') else '소계')

# Map 시도 values based on row ranges
row_to_sido_mapping = {
    (0, 25): '서울특별시',
    (26, 36): '인천광역시',
    (37, 65): '경기도',
    (85, 87): '경기도',
    (66, 84): '강원특별자치도',
    (88, 93): '대전광역시',
    (94, 104): '충청북도',
    (121, 121): '충청북도',
    (105, 119): '충청남도',
    (122, 122): '충청남도',
    (120, 120): '세종특별자치시',
    (123, 139): '부산광역시',
    (140, 148): '대구광역시',
    (149, 154): '울산광역시',
    (155, 178): '경상북도',
    (177, 197): '경상남도',
    (198, 203): '광주광역시',
    (204, 217): '전라북도',
    (218, 240): '전라남도',
    (244, 244): '전라북도',
    (241, 243): '제주특별자치도'
}

def assign_sido(row_idx):
    for (start, end), sido in row_to_sido_mapping.items():
        if start <= row_idx <= end:
            return sido
    return '기타'

data1['시도'] = data1.index.map(assign_sido)


# Reorder columns to move '시도' to the first position
columns = ['시도'] + [col for col in data1.columns if col != '시도']
data1 = data1[columns]



data1

In [ ]:
file_path = "종합 시도 추기.csv"  # 저장할 파일 이름과 경로 설정
data1.to_csv(file_path, index=False, encoding='utf-8-sig')  # UTF-8 인코딩으로 저장

In [ ]:
# 데이터 파일 경로
file1_path = '19년 의료미충족율 표준화 지수.csv'
file2_path = '종합 시도 추기.csv'

# 데이터 로드
data1 = pd.read_csv(file1_path)
data2 = pd.read_csv(file2_path)

# 데이터 병합 (시도와 시군구 기준)
merged_data = data2.merge(
    data1[['시도', '시군구', '표준화율 (%)']],
    on=['시도', '시군구'],
    how='left'
)
merged_data = merged_data.rename(columns={'표준화율 (%)': '미충족 의료윻'})
# 결과 저장
merged_data.to_csv('의료데이터 Ver2 2019.csv', index=False, encoding='utf-8-sig')